# 50 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-11-30 15:33:29.368224: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 15:33:32.050681: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 15:33:32.050788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 15:33:32.442359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 15:33:33.286776: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 15:33:33.287378: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
no_of_patients=50

In [3]:
train=sorted(glob('files4/*.edf'))
train=train[:no_of_patients]
train

['files4/S001R06.edf',
 'files4/S002R06.edf',
 'files4/S003R06.edf',
 'files4/S004R06.edf',
 'files4/S005R06.edf',
 'files4/S006R06.edf',
 'files4/S007R06.edf',
 'files4/S008R06.edf',
 'files4/S009R06.edf',
 'files4/S010R06.edf',
 'files4/S011R06.edf',
 'files4/S012R06.edf',
 'files4/S013R06.edf',
 'files4/S014R06.edf',
 'files4/S015R06.edf',
 'files4/S016R06.edf',
 'files4/S017R06.edf',
 'files4/S018R06.edf',
 'files4/S019R06.edf',
 'files4/S020R06.edf',
 'files4/S021R06.edf',
 'files4/S022R06.edf',
 'files4/S023R06.edf',
 'files4/S024R06.edf',
 'files4/S025R06.edf',
 'files4/S026R06.edf',
 'files4/S027R06.edf',
 'files4/S028R06.edf',
 'files4/S029R06.edf',
 'files4/S030R06.edf',
 'files4/S031R06.edf',
 'files4/S032R06.edf',
 'files4/S033R06.edf',
 'files4/S034R06.edf',
 'files4/S035R06.edf',
 'files4/S036R06.edf',
 'files4/S037R06.edf',
 'files4/S038R06.edf',
 'files4/S039R06.edf',
 'files4/S040R06.edf',
 'files4/S041R06.edf',
 'files4/S042R06.edf',
 'files4/S043R06.edf',
 'files4/S0

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [5]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [6]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [7]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [8]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(600000, 74950, 600000, 74950)

In [9]:
print(xtest.iloc[:,-1].values)

[-5.8e-05 -3.7e-05 -1.0e-06 ... -1.3e-05 -2.6e-05 -2.4e-05]


In [10]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000030,-0.000019,-0.000023,-0.000027,-0.000028,-0.000021,-0.000026,-0.000016,-0.000025,-0.000004,...,-0.000010,-0.000007,-0.000016,-0.000004,0.000000,0.000002,-0.000007,-0.000014,-0.000006,-0.000016
599996,-0.000034,-0.000034,-0.000037,-0.000042,-0.000047,-0.000039,-0.000044,-0.000031,-0.000033,-0.000015,...,-0.000014,0.000007,-0.000001,0.000003,0.000001,-0.000001,0.000008,-0.000001,-0.000003,-0.000007
599997,-0.000030,-0.000034,-0.000032,-0.000040,-0.000051,-0.000043,-0.000046,-0.000025,-0.000029,-0.000012,...,-0.000011,0.000002,-0.000005,0.000005,0.000004,0.000002,0.000011,0.000000,0.000004,-0.000006
599998,-0.000032,-0.000024,-0.000030,-0.000035,-0.000034,-0.000031,-0.000032,-0.000013,-0.000027,-0.000005,...,0.000000,0.000002,0.000002,0.000011,0.000018,0.000014,0.000012,0.000011,0.000018,0.000003


In [11]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [12]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [13]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_4465/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_4465/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_4465/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000030,-0.000019,-0.000023,-0.000027,-0.000028,-0.000021,-0.000026,-0.000016,-0.000025,-0.000004,...,-0.000010,-0.000007,-0.000016,-0.000004,0.000000,0.000002,-0.000007,-0.000014,-0.000006,-0.000016
599996,-0.000034,-0.000034,-0.000037,-0.000042,-0.000047,-0.000039,-0.000044,-0.000031,-0.000033,-0.000015,...,-0.000014,0.000007,-0.000001,0.000003,0.000001,-0.000001,0.000008,-0.000001,-0.000003,-0.000007
599997,-0.000030,-0.000034,-0.000032,-0.000040,-0.000051,-0.000043,-0.000046,-0.000025,-0.000029,-0.000012,...,-0.000011,0.000002,-0.000005,0.000005,0.000004,0.000002,0.000011,0.000000,0.000004,-0.000006
599998,-0.000032,-0.000024,-0.000030,-0.000035,-0.000034,-0.000031,-0.000032,-0.000013,-0.000027,-0.000005,...,0.000000,0.000002,0.000002,0.000011,0.000018,0.000014,0.000012,0.000011,0.000018,0.000003


In [14]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [15]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.66209
[1]	validation_0-mlogloss:3.53682
[2]	validation_0-mlogloss:3.45711
[3]	validation_0-mlogloss:3.39559
[4]	validation_0-mlogloss:3.34792
[5]	validation_0-mlogloss:3.30644
[6]	validation_0-mlogloss:3.27538
[7]	validation_0-mlogloss:3.24662
[8]	validation_0-mlogloss:3.22118
[9]	validation_0-mlogloss:3.19438
[10]	validation_0-mlogloss:3.17717
[11]	validation_0-mlogloss:3.15846
[12]	validation_0-mlogloss:3.14407
[13]	validation_0-mlogloss:3.12944
[14]	validation_0-mlogloss:3.11773
[15]	validation_0-mlogloss:3.10499
[16]	validation_0-mlogloss:3.09529
[17]	validation_0-mlogloss:3.08399
[18]	validation_0-mlogloss:3.07484
[19]	validation_0-mlogloss:3.06546
[20]	validation_0-mlogloss:3.05607
[21]	validation_0-mlogloss:3.04749
[22]	validation_0-mlogloss:3.04007
[23]	validation_0-mlogloss:3.03347
[24]	validation_0-mlogloss:3.02555
[25]	validation_0-mlogloss:3.01801
[26]	validation_0-mlogloss:3.00974
[27]	validation_0-mlogloss:3.00424
[28]	validation_0-mlogloss:2.9

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
18750/18750 [==============================] - 16s 838us/step - loss: 2.4486 - val_loss: 2.8820
Epoch 2/10
18750/18750 [==============================] - 15s 825us/step - loss: 1.9359 - val_loss: 2.7118
Epoch 3/10
18750/18750 [==============================] - 36s 2ms/step - loss: 1.6807 - val_loss: 2.7449
Epoch 4/10
18750/18750 [==============================] - 19s 1ms/step - loss: 1.6167 - val_loss: 2.6841
Epoch 5/10
18750/18750 [==============================] - 17s 912us/step - loss: 1.5794 - val_loss: 2.7076
Epoch 6/10
18750/18750 [==============================] - 16s 828us/step - loss: 1.5546 - val_loss: 2.7850
Epoch 7/10
18750/18750 [==============================] - 16s 869us/step - loss: 1.5358 - val_loss: 2.7789
Epoch 8/10
18750/18750 [==============================] - 17s 929us/step - loss: 1.5199 - val_loss: 2.7111
Epoch 9/10
18750/18750 [==============================] - 705s 38ms/step - loss: 1.5082 - val_loss: 2.9171
Epoch 10/10
18750/18750 [================

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

2343/2343 [==============================] - 1s 335us/step
              precision    recall  f1-score   support

           0       0.18      0.14      0.16      1499
           1       0.12      0.08      0.10      1499
           2       0.32      0.26      0.29      1499
           3       0.74      0.67      0.70      1499
           4       0.77      0.72      0.75      1499
           5       0.47      0.25      0.32      1499
           6       0.48      0.66      0.55      1499
           7       0.54      0.60      0.57      1499
           8       0.64      0.63      0.63      1499
           9       0.48      0.42      0.45      1499
          10       0.36      0.20      0.26      1499
          11       0.22      0.45      0.30      1499
          12       0.28      0.12      0.17      1499
          13       0.12      0.09      0.10      1499
          14       0.17      0.08      0.11      1499
          15       0.40      0.40      0.40      1499
          16       0.3

## 0-16

In [16]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [17]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_4465/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_4465/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_4465/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000030,-0.000019,-0.000023,-0.000027,-0.000028,-0.000021,-0.000026,-0.000016,-0.000025,-0.000004,...,-0.000010,-0.000007,-0.000016,-0.000004,0.000000,0.000002,-0.000007,-0.000014,-0.000006,-0.000016
599996,-0.000034,-0.000034,-0.000037,-0.000042,-0.000047,-0.000039,-0.000044,-0.000031,-0.000033,-0.000015,...,-0.000014,0.000007,-0.000001,0.000003,0.000001,-0.000001,0.000008,-0.000001,-0.000003,-0.000007
599997,-0.000030,-0.000034,-0.000032,-0.000040,-0.000051,-0.000043,-0.000046,-0.000025,-0.000029,-0.000012,...,-0.000011,0.000002,-0.000005,0.000005,0.000004,0.000002,0.000011,0.000000,0.000004,-0.000006
599998,-0.000032,-0.000024,-0.000030,-0.000035,-0.000034,-0.000031,-0.000032,-0.000013,-0.000027,-0.000005,...,0.000000,0.000002,0.000002,0.000011,0.000018,0.000014,0.000012,0.000011,0.000018,0.000003


In [18]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [19]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.57532
[1]	validation_0-mlogloss:3.42308
[2]	validation_0-mlogloss:3.32399
[3]	validation_0-mlogloss:3.23828
[4]	validation_0-mlogloss:3.16996
[5]	validation_0-mlogloss:3.11928
[6]	validation_0-mlogloss:3.07446
[7]	validation_0-mlogloss:3.03235
[8]	validation_0-mlogloss:2.99824
[9]	validation_0-mlogloss:2.96821
[10]	validation_0-mlogloss:2.93868
[11]	validation_0-mlogloss:2.91499
[12]	validation_0-mlogloss:2.89261
[13]	validation_0-mlogloss:2.87120
[14]	validation_0-mlogloss:2.85247
[15]	validation_0-mlogloss:2.83567
[16]	validation_0-mlogloss:2.82070
[17]	validation_0-mlogloss:2.80697
[18]	validation_0-mlogloss:2.78955
[19]	validation_0-mlogloss:2.77647
[20]	validation_0-mlogloss:2.76333
[21]	validation_0-mlogloss:2.74869
[22]	validation_0-mlogloss:2.73260
[23]	validation_0-mlogloss:2.72029
[24]	validation_0-mlogloss:2.70962
[25]	validation_0-mlogloss:2.69838
[26]	validation_0-mlogloss:2.68812
[27]	validation_0-mlogloss:2.67651
[28]	validation_0-mlogloss:2.6

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
18750/18750 [==============================] - 19s 984us/step - loss: 1.6897 - val_loss: 2.6328
Epoch 2/10
18750/18750 [==============================] - 15s 814us/step - loss: 1.0103 - val_loss: 2.7506
Epoch 3/10
18750/18750 [==============================] - 16s 850us/step - loss: 0.8799 - val_loss: 2.8169
Epoch 4/10
18750/18750 [==============================] - 15s 811us/step - loss: 0.8145 - val_loss: 2.8397
Epoch 5/10
18750/18750 [==============================] - 15s 812us/step - loss: 0.7716 - val_loss: 3.0011
Epoch 6/10
18750/18750 [==============================] - 15s 816us/step - loss: 0.7403 - val_loss: 3.0550
Epoch 7/10
18750/18750 [==============================] - 15s 820us/step - loss: 0.7113 - val_loss: 2.9857
Epoch 8/10
18750/18750 [==============================] - 15s 818us/step - loss: 0.6854 - val_loss: 2.9270
Epoch 9/10
18750/18750 [==============================] - 16s 845us/step - loss: 0.6628 - val_loss: 2.9154
Epoch 10/10
18750/18750 [============

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

2343/2343 [==============================] - 1s 349us/step
              precision    recall  f1-score   support

           0       0.40      0.42      0.41      1499
           1       0.14      0.15      0.15      1499
           2       0.45      0.30      0.36      1499
           3       0.70      0.50      0.59      1499
           4       0.91      0.80      0.85      1499
           5       0.63      0.28      0.39      1499
           6       0.57      0.85      0.68      1499
           7       0.84      0.59      0.69      1499
           8       0.59      0.64      0.62      1499
           9       0.77      0.50      0.61      1499
          10       0.54      0.40      0.46      1499
          11       0.56      0.43      0.49      1499
          12       0.43      0.51      0.47      1499
          13       0.26      0.30      0.28      1499
          14       0.25      0.24      0.24      1499
          15       0.66      0.49      0.56      1499
          16       0.0

## 0-32

In [20]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [21]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_4465/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_4465/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_4465/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [22]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [23]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.43197
[1]	validation_0-mlogloss:3.21146
[2]	validation_0-mlogloss:3.07842
[3]	validation_0-mlogloss:2.97043
[4]	validation_0-mlogloss:2.89345
[5]	validation_0-mlogloss:2.82348
[6]	validation_0-mlogloss:2.75691
[7]	validation_0-mlogloss:2.70334
[8]	validation_0-mlogloss:2.65758
[9]	validation_0-mlogloss:2.61606
[10]	validation_0-mlogloss:2.57477
[11]	validation_0-mlogloss:2.53175
[12]	validation_0-mlogloss:2.49910
[13]	validation_0-mlogloss:2.46778
[14]	validation_0-mlogloss:2.43974
[15]	validation_0-mlogloss:2.41224
[16]	validation_0-mlogloss:2.38896
[17]	validation_0-mlogloss:2.36210
[18]	validation_0-mlogloss:2.34049
[19]	validation_0-mlogloss:2.31448
[20]	validation_0-mlogloss:2.29445
[21]	validation_0-mlogloss:2.27922
[22]	validation_0-mlogloss:2.26131
[23]	validation_0-mlogloss:2.24316
[24]	validation_0-mlogloss:2.22098
[25]	validation_0-mlogloss:2.20459
[26]	validation_0-mlogloss:2.19083
[27]	validation_0-mlogloss:2.17829
[28]	validation_0-mlogloss:2.1

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
18750/18750 [==============================] - 18s 932us/step - loss: 1.1595 - val_loss: 2.6192
Epoch 2/10
18750/18750 [==============================] - 17s 932us/step - loss: 0.4525 - val_loss: 2.8171
Epoch 3/10
18750/18750 [==============================] - 17s 904us/step - loss: 0.3301 - val_loss: 2.7147
Epoch 4/10
18750/18750 [==============================] - 17s 925us/step - loss: 0.2749 - val_loss: 2.8720
Epoch 5/10
18750/18750 [==============================] - 19s 1ms/step - loss: 0.2407 - val_loss: 2.7542
Epoch 6/10
18750/18750 [==============================] - 17s 909us/step - loss: 0.2180 - val_loss: 2.4409
Epoch 7/10
18750/18750 [==============================] - 17s 924us/step - loss: 0.2019 - val_loss: 2.5652
Epoch 8/10
18750/18750 [==============================] - 17s 930us/step - loss: 0.1867 - val_loss: 2.5334
Epoch 9/10
18750/18750 [==============================] - 17s 930us/step - loss: 0.1753 - val_loss: 2.5839
Epoch 10/10
18750/18750 [==============

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

2343/2343 [==============================] - 1s 360us/step
              precision    recall  f1-score   support

           0       0.83      0.62      0.71      1499
           1       0.30      0.20      0.24      1499
           2       0.65      0.86      0.74      1499
           3       0.66      0.34      0.45      1499
           4       0.91      0.95      0.93      1499
           5       0.96      0.77      0.85      1499
           6       0.77      0.94      0.85      1499
           7       0.77      0.38      0.51      1499
           8       0.57      0.94      0.71      1499
           9       0.76      0.67      0.71      1499
          10       0.57      0.90      0.70      1499
          11       0.97      0.80      0.88      1499
          12       0.73      0.85      0.78      1499
          13       0.56      0.82      0.67      1499
          14       0.59      0.54      0.56      1499
          15       0.94      0.72      0.81      1499
          16       0.4

## 0-64

In [24]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [25]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [26]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.17302
[1]	validation_0-mlogloss:2.88248
[2]	validation_0-mlogloss:2.70533
[3]	validation_0-mlogloss:2.57009
[4]	validation_0-mlogloss:2.45687
[5]	validation_0-mlogloss:2.36547
[6]	validation_0-mlogloss:2.28165
[7]	validation_0-mlogloss:2.20928
[8]	validation_0-mlogloss:2.15253
[9]	validation_0-mlogloss:2.09348
[10]	validation_0-mlogloss:2.04743
[11]	validation_0-mlogloss:2.00298
[12]	validation_0-mlogloss:1.96501
[13]	validation_0-mlogloss:1.92658
[14]	validation_0-mlogloss:1.88770
[15]	validation_0-mlogloss:1.85190
[16]	validation_0-mlogloss:1.81301
[17]	validation_0-mlogloss:1.78710
[18]	validation_0-mlogloss:1.76009
[19]	validation_0-mlogloss:1.73157
[20]	validation_0-mlogloss:1.70950
[21]	validation_0-mlogloss:1.69024
[22]	validation_0-mlogloss:1.66858
[23]	validation_0-mlogloss:1.64575
[24]	validation_0-mlogloss:1.62617
[25]	validation_0-mlogloss:1.60643
[26]	validation_0-mlogloss:1.58944
[27]	validation_0-mlogloss:1.57376
[28]	validation_0-mlogloss:1.5

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
18750/18750 [==============================] - 19s 984us/step - loss: 0.8086 - val_loss: 2.7155
Epoch 2/10
18750/18750 [==============================] - 293s 16ms/step - loss: 0.2965 - val_loss: 2.6128
Epoch 3/10
18750/18750 [==============================] - 19s 1ms/step - loss: 0.2116 - val_loss: 2.5895
Epoch 4/10
18750/18750 [==============================] - 17s 896us/step - loss: 0.1673 - val_loss: 2.4293
Epoch 5/10
18750/18750 [==============================] - 17s 906us/step - loss: 0.1415 - val_loss: 2.2409
Epoch 6/10
18750/18750 [==============================] - 17s 902us/step - loss: 0.1226 - val_loss: 2.4476
Epoch 7/10
18750/18750 [==============================] - 17s 904us/step - loss: 0.1101 - val_loss: 2.2728
Epoch 8/10
18750/18750 [==============================] - 16s 841us/step - loss: 0.1007 - val_loss: 2.2738
Epoch 9/10
18750/18750 [==============================] - 14s 769us/step - loss: 0.0926 - val_loss: 2.2116
Epoch 10/10
18750/18750 [==============

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

2343/2343 [==============================] - 1s 309us/step
              precision    recall  f1-score   support

           0       0.83      0.76      0.79      1499
           1       0.62      0.39      0.47      1499
           2       0.70      0.87      0.78      1499
           3       0.88      0.82      0.85      1499
           4       0.88      1.00      0.93      1499
           5       0.91      0.97      0.94      1499
           6       0.91      0.83      0.87      1499
           7       0.81      0.49      0.62      1499
           8       0.73      0.96      0.83      1499
           9       0.96      0.92      0.94      1499
          10       0.62      0.53      0.57      1499
          11       0.87      0.94      0.90      1499
          12       0.91      0.78      0.84      1499
          13       0.67      0.57      0.62      1499
          14       0.79      0.73      0.76      1499
          15       0.86      0.88      0.87      1499
          16       0.8